In [39]:
import pandas as pd
import boto3
import json

In [40]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('conf.cfg'))

KEY = config.get('AWS', 'AWS_ACCESS_KEY')
SECRET = config.get('AWS', 'AWS_ACCESS_SECRET')


CLUSTER_TYPE = config.get('CLUSTER', 'CLUSTER_TYPE')
NUM_NODES = config.get('CLUSTER', 'NUM_NODES')
NODE_TYPE = config.get('CLUSTER', 'NODE_TYPE')

CLUSTER_ID = config.get('CLUSTER', 'CLUSTER_ID')
DB_NAME = config.get('CLUSTER', 'DB_NAME')
DB_USER = config.get('CLUSTER', 'DB_USER')
DB_PASSWORD = config.get('CLUSTER', 'DB_PASSWORD')
DB_PORT = config.get('CLUSTER', 'DB_PORT')

IAM_ROLE_NAME = config.get('IAM_ROLE', 'IAM_ROLE_NAME')


(DB_USER, DB_PASSWORD, DB_NAME)

pd.DataFrame({"Param":
                  ["CLUSTER_TYPE", "NUM_NODES", "NODE_TYPE", "CLUSTER_IDENTIFIER", "DB_NAME", "DB_USER", "DB_PASSWORD", "DB_PORT", "IAM_ROLE_NAME"],
              "Value":
                  [CLUSTER_TYPE, NUM_NODES, NODE_TYPE, CLUSTER_ID, DB_NAME, DB_USER, DB_PASSWORD, DB_PORT, IAM_ROLE_NAME]
             })

,Param,Value
0,CLUSTER_TYPE,multi-node
1,NUM_NODES,4
2,NODE_TYPE,dc2.large
3,CLUSTER_IDENTIFIER,awsClusters
4,DB_NAME,dev
5,DB_USER,awsuser
6,DB_PASSWORD,Passw0rd
7,DB_PORT,5439
8,IAM_ROLE_NAME,capstoneRole


In [41]:
iam = boto3.client('iam', aws_access_key_id = KEY,
                  aws_secret_access_key = SECRET,
                  region_name = "us-west-2"
                  )

redshift = boto3.client('redshift', 
                       region_name = "us-west-2",
                       aws_access_key_id = KEY,
                       aws_secret_access_key=SECRET
                       )

In [42]:
from botocore.exceptions import ClientError

#1.1 Create the role, 
try:
    print("1.1 Creating a new IAM Role") 
    dbRole = iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole', 
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )
    print("1.1.2 Role created...")
except Exception as e:
    print(e)
    
    
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonRedshiftFullAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.1 Creating a new IAM Role
1.1.2 Role created...
1.2 Attaching Policy
1.3 Get the IAM role ARN
arn:aws:iam::999027378457:role/capstoneRole


In [43]:
# Resume cluster if already created
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=int(NUM_NODES),

        #Identifiers & Credentials
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_ID,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
except Exception as e:
    print(e)

In [48]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_ID)['Clusters'][0]
prettyRedshiftProps(myClusterProps)



C:\Users\ajayi\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Key,Value
0,ClusterIdentifier,awsclusters
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,awsuser
4,DBName,dev
5,Endpoint,"{'Address': 'awsclusters.c3rcwbdmkxwc.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-757b2d0d
7,NumberOfNodes,4


In [49]:
ENDPOINT = myClusterProps['Endpoint']['Address']
ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", ENDPOINT)
print("DWH_ROLE_ARN :: ", ROLE_ARN)

DWH_ENDPOINT ::  awsclusters.c3rcwbdmkxwc.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::999027378457:role/capstoneRole


In [50]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DB_PORT),
        ToPort=int(DB_PORT)
    )
except Exception as e:
    print(e)

name 'ec2' is not defined
